# 1.3 Transformación de datos

Se usa la librería pandas para abrir el archivo csv extraído de la base de datos para su análisis y posterior transformación

In [1]:
import pandas as pd

In [2]:
df_db = pd.read_csv("data_prueba_extraida.csv")
df_orig = pd.read_csv("../dataset/data_prueba_tecnica.csv")

In [3]:
df_db.head()

,id,company_name,company_id,amount,status,created_at,updated_at
0,48ba4bdbfb56ceebb32f2bd0263e759be942af3d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-03-19,NaN
1,05fc6f5ac66b6ee7e4253aa5d0c2299eb47aaaf4,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,pending_payment,2019-05-06,NaN
2,2cdce231c1fc6a2061bfa2f1d978351fe217245d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-02-22,NaN
3,81633ba310a50b673efd469c37139576982901aa,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,6ccfc4c24e788e4bca448df343698782db6b0c0b,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05


Bajo este resumen se deduce que hay valores tipo string para los ids, nombre de compañia, status y fechas. En cambio amount es de tipo numérico, por lo que lo comprobaremos usando python. La primera impresión es de que probablemente el id tenga un tamaño que rebase los 24 que exige que se guarden en la base de datos.

In [4]:
df_db.dtypes

id               object
company_name     object
company_id       object
amount          float64
status           object
created_at       object
updated_at       object
dtype: object

In [5]:
df_db.shape

(10000, 7)

Transformamos los datos en tipo str para facilitar su transformación a excepción de los que sean de tipo numéricos

In [6]:
for column in df_db:
    if df_db[column].dtype == "object":
        df_db[column] = df_db[column].astype(str)

Veamos cuántos caracteres tiene el id más grande.

In [9]:
df_db['id'].apply(len).max

<bound method Series.max of 0       40
1       40
2       40
3       40
4       40
        ..
9995    40
9996    40
9997    40
9998    40
9999    40
Name: id, Length: 10000, dtype: int64>

También se identificó que los ids son presumiblemente números en formato hexadecimal guardados como strings, puede que sean valores comprimidos o encriptados, por lo que se comprobará si un posible extracción de substring para que se adapte a la nueva base de datos donde exigen que tenga 24 caracteres no cree ids duplicados.

In [7]:
len(list(df_db["id"].apply(lambda x: x[16:]).unique()))

9998

Se determina que posiblemente hay al menos dos id son duplicados. Lo comprobamos sin quitarle caracteres.

In [8]:
len(list(df_db["id"].unique()))

9998

Se deduce que hay al menos dos datos que pueden estar duplicados, ahora veamos como se comportan los demás datos. El esquema exige que el valor máximo de amount sea de 10 dígitos y dos decimales. Veamos cuál es el valor más grande:

In [10]:
df_db['amount'].max()

2.9999999999999997e+34

Se nota que existe un valor que rompe con la lógica teniendo un exponente de hasta 34 en base 10. Haré un filtro que todo valor corrupto sea 0.0 si sobrepasa el umbral que requiere el esquema de base de datos.

In [ ]:
df_db['amount'] = df_db['amount'].where(df['amount'] < pow(10, 16), 0.0)

Ahora veamos los nombres de las compañias y cúantas son en total.

In [11]:
df_db["company_name"].unique()

array(['MiPasajefy', 'Muebles chidos', 'nan', 'MiPas0xFFFF', 'MiP0xFFFF'],
      dtype=object)

Aquí se notó que hay al menos 2 datos que pueden estar corruptos y uno que presumiblemente puede referirse a un valor nulo. Veamos cómo se comporta las filas dado estos datos.

In [14]:
df_db.loc[df_db['company_name'] == "nan"]

,id,company_name,company_id,amount,status,created_at,updated_at
731,ec79a21ef969c7fc6beef080ff56baf0aeeca8b5,nan,cbf1c8b09cd5b549416d49d220a40cbd317f952e,112.52,paid,2019-02-14,2019-02-14
2199,6a6ac16d53a02ba7948bff0a534e45404e716c5b,nan,cbf1c8b09cd5b549416d49d220a40cbd317f952e,244.88,voided,2019-05-09,nan
2200,4740cf2624c3b929d9944cdfcb5c87e71a82ddc5,nan,cbf1c8b09cd5b549416d49d220a40cbd317f952e,118.78,paid,2019-03-05,2019-03-06


In [15]:
df_db.loc[df_db['company_name'] == "MiPas0xFFFF"]

,id,company_name,company_id,amount,status,created_at,updated_at
1320,60366f3cfb5245ec62143984eef4ba93cfbe3c38,MiPas0xFFFF,cbf1c8b09cd5b549416d49d220a40cbd317f952e,134.99,paid,2019-01-25,2019-01-25


In [19]:
df_db["company_id"].loc[df_db['company_name'] == 'MiP0xFFFF'].values[0]

'cbf1c8b09cd5b549416d49d220a40cbd317f952e'

Como company_id y company_name tiene posibilidad de que esten relacionados, vamos a ver sus dataframe aislado, eliminando valores duplicados.

In [18]:
df_companies = df_db[["company_id", "company_name"]]
df_companies_wout_dup = df_companies.drop_duplicates()
df_companies_wout_dup.head()

,company_id,company_name
0,cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiPasajefy
78,8f642dc67fccf861548dfe1c761ce22f795e91f0,Muebles chidos
262,nan,MiPasajefy
603,*******,MiPasajefy
731,cbf1c8b09cd5b549416d49d220a40cbd317f952e,nan


Veamos cuántos company_id hay diferentes

In [21]:
df_companies_id = df_db[["company_id"]].drop_duplicates()
df_companies_id.shape

(4, 1)

In [ ]:
df_companies_id.head()

,company_id
0,cbf1c8b09cd5b549416d49d220a40cbd317f952e
78,8f642dc67fccf861548dfe1c761ce22f795e91f0
262,nan
603,*******


In [7]:
df_orig["name"].unique()

array(['MiPasajefy', 'Muebles chidos', nan, 'MiPas0xFFFF', 'MiP0xFFFF'],
      dtype=object)

In [ ]:
df_orig["company_id"].unique()

9998

In [ ]:
df_db['id'].apply(len).max

df_db['id'] = df_db['id'].apply(lambda x: x[16:])

df_db['id'].apply(len).max

<bound method Series.max of 0       24
1       24
2       24
3       24
4       24
        ..
9995    24
9996    24
9997    24
9998    24
9999    24
Name: id, Length: 10000, dtype: int64>

,company_id,company_name
0,cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiPasajefy
78,8f642dc67fccf861548dfe1c761ce22f795e91f0,Muebles chidos
262,NaN,MiPasajefy
603,*******,MiPasajefy
731,cbf1c8b09cd5b549416d49d220a40cbd317f952e,NaN
